In [1]:
"""
Trabalho computacional sobre algoritmos evolutivos para solução de sistemas de equações não lineares 
"""

'\nTrabalho computacional sobre algoritmos evolutivos para solução de sistemas de equações não lineares \n'

In [2]:
"""
Sistema de equações não lineares
"""
import numpy as np
import random


class EquationSystem:
    def __init__(self, equations):
        self.equations = equations

    def evaluate(self, x):
        return [np.abs(equation(x)) for equation in self.equations]


p1 = EquationSystem([
    lambda x: 0.8*(x[0]**2 + x[0] -1)*x[2] + 0.12*x[0]**2 + 2.16*x[0] - 0.12,
    lambda x: (1 + x[0]**2)*x[3] + 0.4*x[0]**2 - 1.6*x[0] - 0.4,
    lambda x: (1 + x[0]**2)*x[4] + x[0]**2 - 1,
    lambda x: (1 + x[0]**2)*x[5] + 0.8*(x[0]**2 + x[0] -1),
    lambda x: x[4]*x[6] - 0.02*x[5] - x[4] - x[2]*x[3] - 0.16*x[0],
    lambda x: x[6]**2 - 2*x[3]*x[6] + x[5]**2 + x[3]**2 - x[1]**2,
    lambda x: x[7] - x[1]*x[2],
    lambda x: 0.0476*x[2]*x[7]**12 + x[2] - 2.104
])

# p1 bounds
p1_lb = [-3, -1, -2, -1, -1, -0.5, -1.5, -1.5]
p1_ub = [1, 1, 2, 1, 1, 0.5, 1.5, 1.5]

In [3]:
p2 = EquationSystem([
    lambda x: x[4] + x[3] - 1.803,
    lambda x: (x[1] + x[2])*x[4] + 6.19116*x[3] - 1.803*(1.497 + 0.035),
    lambda x: x[5] + x[3] -0.328,
    lambda x: 0.28801*x[5] - x[1]*x[2]*x[4],
    lambda x: (-6.19116*x[0] + x[0]*x[2] + x[1]*x[4] - x[2]*x[4])*x[5] + x[0]*x[2]*x[4],
    lambda x: 1.571*x[6] + x[3] -1.803,
    lambda x: x[7] - 0.000856*x[6]**2,
    lambda x: (x[4] -x[0])*x[8] - x[0]*x[4],
    lambda x: x[8] - 377*x[1]*x[7]
])

# p2 bounds
p2_lb = [-0.5, -1, -1, -1, 1, -1, -1, 0, -1]
p2_ub = [0.5, 1, 1, 1, 2, 1, 1, 1, 1]

In [4]:
# Real-Coded Genetic Algorithm

def generate_individual(upper_bound: list, lower_bound: list) -> np.array:
    return np.array(
        [
            random.uniform(lower_bound[i], upper_bound[i])
            for i in range(len(upper_bound))
        ]
    )


def generate_population(upper_bound: list, lower_bound: list, pop_size) -> list:
    return [generate_individual(upper_bound, lower_bound) for _ in range(pop_size)]


# evaluate Using the penalty method
def evaluate_penalty(individual, equation_system):
    result = equation_system.evaluate(individual)
    penalty_sum = 0
    for r in result:
        penalty_sum += max(0, r) ** 2
    return penalty_sum

"""
 Um exemplo específico para o caso de codificação em ponto
flutuante é o chamado crossover aritmético (MICHALEWICZ, 1996). Este operador é
definido como uma fusão de dois vetores (cromossomos): se x1 e x2 são dois indivíduos
selecionados para crossover, os dois filhos resultantes serão x_1^line = alpha*x_1 + (1 - alpha)*x_2 e
x_2^line = (1 - alpha)*x_1 + alpha*x_2, sendo alpha um número aleatório pertencente ao intervalo [0, 1]. 
"""

def arithmetical_crossover(parent1, parent2, alpha):
    parent1, _ = parent1
    parent2, _ = parent2
    offspring1 = alpha * parent1 + (1 - alpha) * parent2
    offspring2 = (1 - alpha) * parent1 + alpha * parent2
    return (offspring1, None), (offspring2, None)

"""
o caso de problemas com codificação em ponto flutuante, o operador de mutação
mais popular é a mutação gaussiana (MICHALEWICZ & S CHOENAUER , 1996), modificando
todos os componentes de um cromossomo x = [x1 … xn] na forma:
x^line = x + N(0, sigma)

Sendo N(0, sigma) um vetor de variáveis aleatórias independentes, com distribuição normal,
média zero e desvio padrão sigma
"""

def gaussian_mutation(individual, sigma):
    ind, fitness = individual
    return ind + np.random.normal(0, sigma, len(ind)), fitness


def roulette_wheel_selection(evaluated_population):
    total_fitness = 0
    for individual, fitness in evaluated_population:
        total_fitness += 1/fitness
    r = random.uniform(0, total_fitness)
    acc = 0
    for individual, fitness in evaluated_population:
        acc += 1/fitness
        if acc >= r:
            return individual, fitness
    return evaluated_population[-1]


def real_coded_genetic_algorithm(equation_system, lower_bound, upper_bound, pop_size, max_gen, pc, pm, alpha, sigma, elitisim=True):
    """
    Real-Coded Genetic Algorithm
    :param equation_system: EquationSystem
    :param lower_bound: list
    :param upper_bound: list
    :param pop_size: int
    :param max_gen: int
    :param pc: float - Crossover probability
    :param pm: float - Mutation probability
    :param alpha: float - Crossover parameter
    :param sigma: float - Mutation parameter
    """
    # pop_size must be even
    if pop_size % 2 != 0:
        raise ValueError("pop_size must be even")

    population = generate_population(upper_bound, lower_bound, pop_size)
    population = [(individual, evaluate_penalty(individual, equation_system)) for individual in population]

    history = []
    for gen in range(max_gen):
        
        print(f"Generation {gen+1}", end="\r")
        # Evaluate
        evaluated_population = []
        # print(population)
        for individual, _ in population:
            fitness = evaluate_penalty(individual, equation_system)
            evaluated_population.append((individual, fitness))
        evaluated_population = sorted(evaluated_population, key=lambda x: x[1])

        # Keep 1 elite individual
        if elitisim:
            elite = evaluated_population[0]
        else:
            elite = None
        
        # Selection
        selected_population = []
        for _ in range(pop_size):
            selected_population.append(roulette_wheel_selection(evaluated_population))
        # Crossover
        offspring_population = []
        for i in range(0, pop_size, 2):
            if random.random() < pc:
                # print(selected_population[i], selected_population[i+1])
                offspring1, offspring2 = arithmetical_crossover(selected_population[i], selected_population[i+1], alpha)
                offspring_population.append(offspring1)
                offspring_population.append(offspring2)
            else:
                offspring_population.append(selected_population[i])
                offspring_population.append(selected_population[i+1])

        # Mutation
        mutated_population = []
        for individual in offspring_population:
            if random.random() < pm:
                mutated_population.append(gaussian_mutation(individual, sigma))
            else:
                mutated_population.append(individual)

        # Replace population
        population = mutated_population
        if elite:
            population[0] = elite
        
        best_fit = evaluated_population[0][1]
        history.append(best_fit)

    return evaluated_population[0], history


In [5]:
result, history = real_coded_genetic_algorithm(p1, p1_lb, p1_ub, 100, 2000, 0.8, 0.1, 0.5, 0.1, elitisim=True)
best_individual, best_fitness = result

In [6]:
result

(array([0.29102461, 1.1738793 , 1.08905194, 0.65018367, 0.88349306,
        0.42025754, 1.75249159, 1.28308001]),
 0.030536439248830465)

In [7]:
# não tá tudo dando 0 ainda, tem q melhorar o algoritmo
p1.evaluate(best_individual)

[0.025122120384959423,
 0.1265100625193652,
 0.043016118213664134,
 0.04357265967313673,
 0.09823178305081254,
 0.01370652239377601,
 0.004664475311973337,
 0.017101169929816695]

In [8]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=history, mode='lines'))
fig.show()

In [9]:
# evolution strategy (mu, lambda) of the ackley objective function
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed

# penalty evaluation function
def evaluate_penalty(individual, equation_system):
    result = equation_system.evaluate(individual)
    penalty_sum = 0
    for r in result:
        penalty_sum += max(0, np.abs(r)) ** 2
    return penalty_sum

# check if a point is within the bounds of the search
def in_bounds(point, bounds):
    # enumerate all dimensions of the point
    for d in range(len(bounds)):
        # check if out of bounds for this dimension
        if point[d] < bounds[d, 0] or point[d] > bounds[d, 1]:
            return False
    return True

# evolution strategy (mu, lambda) algorithm
def es_comma(objective, bounds, n_iter, step_size, mu, lam):
    best, best_eval = None, 1e+10
    # calculate the number of children per parent
    n_children = int(lam / mu)
    # initial population
    population = list()
    for _ in range(lam):
        candidate = None
        while candidate is None or not in_bounds(candidate, bounds):
            candidate = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
        population.append(candidate)
    # perform the search
    for epoch in range(n_iter):
        # evaluate fitness for the population
        scores = [objective(c) for c in population]
        # rank scores in ascending order
        ranks = np.argsort(np.argsort(scores))
        # select the indexes for the top mu ranked solutions
        selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]
        # create children from parents
        children = list()
        for i in selected:
            # check if this parent is the best solution ever seen
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
                print('Epoch %d, Best: f(%s) = %.5f' % (epoch, best, best_eval))
            # create children for parent
            for _ in range(n_children):
                child = None
                while child is None or not in_bounds(child, bounds):
                    child = population[i] + randn(len(bounds)) * step_size
                children.append(child)
        # replace population with children
        population = children
    return [best, best_eval]


In [10]:
# seed the pseudorandom number generator
seed(1)
# define range for input
p1_lb = [-3, -1, -2, -1, -1, -0.5, -1.5, -1.5]
p1_ub = [1, 1, 2, 1, 1, 0.5, 1.5, 1.5]
# bounds = (min, max) for each variable
bounds = np.asarray([[p1_lb[i], p1_ub[i]] for i in range(len(p1_lb))])

print(bounds)
# define the total iterations
n_iter = 5000

# define the maximum step size
step_size = 0.15

# number of parents selected
mu = 20

# the number of children generated by parents
lam = 100

# perform the evolution strategy (mu, lambda) search
best, score = es_comma(lambda x: evaluate_penalty(x, p1), bounds, n_iter, step_size, mu, lam)
print('Done!')
print('f(%s) = %f' % (best, score))

p1.evaluate(best)

[[-3.   1. ]
 [-1.   1. ]
 [-2.   2. ]
 [-1.   1. ]
 [-1.   1. ]
 [-0.5  0.5]
 [-1.5  1.5]
 [-1.5  1.5]]
Epoch 0, Best: f([-0.70352958 -0.70654285  0.35722215  0.39951672 -0.79533114 -0.08594401
  0.58320047 -0.25746219]) = 13.44354
Epoch 0, Best: f([ 0.53322436  0.24734441  1.00376974 -0.30220332 -0.46014422  0.39588622
 -0.21572643  1.39452014]) = 9.13644
Epoch 0, Best: f([ 0.80070448  0.11330638  1.6624254   0.28313242 -0.21998457 -0.01400933
  0.31293145  0.14864376]) = 7.11733
Epoch 0, Best: f([ 0.45791772 -0.35463801  0.68315516 -0.09825213 -0.2357945  -0.08918865
 -0.29556125 -0.54784816]) = 5.37675
Epoch 0, Best: f([-0.08459649  0.03337757  0.82782498  0.56105911 -0.25024811  0.27032253
  0.75187296  0.33963363]) = 4.91366
Epoch 0, Best: f([ 0.1783134   0.01415985  1.16728416  0.39152843  0.55569695 -0.09351713
  0.4433119  -0.96061711]) = 3.37643
Epoch 1, Best: f([ 4.79037870e-01  5.78720126e-02  1.65980716e+00  3.92842795e-01
 -7.68102217e-02  3.52682763e-04  4.67949069e-01  

[0.0067086350524167004,
 0.18711903608227015,
 0.0005079318485543549,
 0.04825930494965014,
 0.27207061546393957,
 0.06986545819587708,
 0.2328739599277747,
 0.0639687062654879]

In [11]:
result, history = real_coded_genetic_algorithm(p2, p2_lb, p2_ub, 100, 2000, 0.8, 0.1, 0.5, 0.1, elitisim=True)
best_individual, best_fitness = result

In [12]:
p2.evaluate(best_individual)

[0.01142281048569549,
 0.01746919306088479,
 0.04353536728329338,
 0.02204979483551867,
 0.028496124070938246,
 0.007859020457853028,
 0.017738352601652962,
 0.03861518635597316,
 0.009888020860286922]

In [13]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=history, mode='lines'))
fig.show()

In [14]:
# seed the pseudorandom number generator
seed(1)

# bounds = (min, max) for each variable
bounds = np.asarray([[p2_lb[i], p2_ub[i]] for i in range(len(p2_lb))])

print(bounds)
# define the total iterations
n_iter = 5000

# define the maximum step size
step_size = 0.15

# number of parents selected
mu = 20

# the number of children generated by parents
lam = 100

# perform the evolution strategy (mu, lambda) search
best, score = es_comma(lambda x: evaluate_penalty(x, p2), bounds, n_iter, step_size, mu, lam)
print('Done!')
print('f(%s) = %f' % (best, score))

p2.evaluate(best)

[[-0.5  0.5]
 [-1.   1. ]
 [-1.   1. ]
 [-1.   1. ]
 [ 1.   2. ]
 [-1.   1. ]
 [-1.   1. ]
 [ 0.   1. ]
 [-1.   1. ]]
Epoch 0, Best: f([ 0.11714491 -0.3467102   0.0541162   0.7718842   1.35726976  0.8170703
  0.24672023  0.01582124  0.85887447]) = 15.23641
Epoch 1, Best: f([ 0.37217314 -0.25539265 -0.22809543  0.84647456  1.39286966  0.49540369
  0.19128587  0.013203    0.96858485]) = 11.89274
Epoch 1, Best: f([ 0.22620534  0.35837884 -0.24517445  0.38701334  1.53168438  0.53259313
 -0.57540816  0.00522938 -0.40817627]) = 8.01692
Epoch 1, Best: f([ 0.04656261  0.74352006  0.17633981  0.52651145  1.29896312 -0.21347352
 -0.2668183   0.00246976 -0.24726463]) = 6.82121
Epoch 2, Best: f([-2.39346379e-01 -3.04908652e-01  3.16536352e-01  5.08302312e-01
  1.43434703e+00  9.88251029e-01  1.51813846e-01  1.41563369e-03
  6.91285524e-01]) = 5.98097
Epoch 2, Best: f([-0.09310192  0.01136398  0.29464535  0.02830886  1.75352167 -0.45055609
  0.80608473  0.06247845  0.3964216 ]) = 5.87110
Epoch 3, B

[0.14224520633401627,
 0.10742053645791083,
 0.011116549266888043,
 0.01606547045707718,
 0.059327870033087954,
 0.003882224380658883,
 0.003944987613633429,
 0.14294041369992913,
 0.019887446679502152]